In [1]:
from aocd import get_data
import numpy as np

- Create a 2D grid representing the map
- Identify the guard's starting position and initial facing direction (`^`, `>`, `v`, or `<`)
- If the guard faces an obstacle (`#`) in the direction they are heading, turn 90 degrees to the right
  -  Otherwise, move one step forward in the current facing direction
- Track and store all distinct positions visited by the guard, including the starting position
- Continue moving according to the rules until the guard moves outside the bounds of the grid
- Count the number of distinct positions the guard visited
- Return the total count of distinct positions

In [2]:
direction_map = {
        '^': (-1, 0),
        '>': (0, 1),
        'v': (1, 0),
        '<': (0, -1),
    }

In [3]:
# to be used like turn_right[direction]
turn_right = {'^': '>', '>': 'v', 'v': '<', '<': '^'}

In [4]:
def is_within_bounds(pos, grid):
    return 0 <= pos[0] < len(grid) and 0 <= pos[1] < len(grid[0])
    
def hits_obstacle(pos, grid):
    return grid[pos[0]][pos[1]] == '#'

In [22]:
def simulate_patrol(grid, start_pos, start_dir):
    visited = set()
    pos, direction = start_pos, start_dir
    rows, cols = grid.shape

    visited.add((pos, direction))
       
    while True:
        delta_r, delta_c = direction_map[direction]
        next_pos = (pos[0] + delta_r, pos[1] + delta_c)

        if not is_within_bounds(next_pos, grid):
            break
            
        elif hits_obstacle(next_pos, grid):
            direction = turn_right[direction]

        else:
            pos = next_pos
            visited.add((pos, direction))
    
    return visited


In [23]:
ex_grid_str = """
....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
"""

In [24]:
example_grid = np.array([list(line) for line in ex_grid_str.strip().split("\n")])

In [25]:
def find_start_pos(grid):
    mask = np.isin(grid, list(direction_map.keys()))
    indices = np.argwhere(mask)
    
    if len(indices) > 0:
        i, j = indices[0]
        return (i, j), grid[i, j]
    else:
        return None, None 

In [26]:
ex_start_pos, ex_start_dir = find_start_pos(example_grid)

In [27]:
ex_start_pos, ex_start_dir

((np.int64(6), np.int64(4)), np.str_('^'))

In [28]:
visited_positions = simulate_patrol(example_grid, ex_start_pos, ex_start_dir)

In [29]:
len(visited_positions)

45

Now for the real data:

In [30]:
grid = np.array([list(line) for line in get_data(day=6, year=2024).strip().split("\n")])

In [31]:
start_pos, start_dir = find_start_pos(grid)

In [32]:
visited_positions = simulate_patrol(grid, start_pos, start_dir)

In [33]:
len(visited_positions)

5141

### Part Two

Follow the guard's movement rules (move forward unless blocked, then turn right) until they exit the grid or repeat a _position and direction_
- Keep track of every (position, direction) pair the guard visits
- If the guard visits the same position and direction more than once, they are in a loop

Identify Loop-Causing Positions
- For each empty position (excluding the starting position), simulate placing an obstruction there
- Re-run the patrol simulation with this obstruction in place
- Check if the guard gets stuck in a loop 
- Count the positions where adding an obstruction causes the guard to enter a loop
- Exclude the guard's starting position


In [19]:
def simulate_patrol(grid, start_pos, start_dir):
    visited = set()
    pos, direction = start_pos, start_dir
    visited.add((pos, direction))
    
    while True:
        # Add detection for looping - this is the only change
        if (pos, direction) in visited:
            return visited  # The guard is looping
        
        delta_r, delta_c = direction_map[direction]
        next_pos = (pos[0] + delta_r, pos[1] + delta_c)

        if not is_within_bounds(next_pos, grid):
            break
        
        elif hits_obstacle(next_pos, grid):
            direction = turn_right[direction]
        
        else:
            pos = next_pos
            visited.add((pos, direction))

def find_loop_positions(grid, start_pos, start_dir):
    potential_positions = []
    rows, cols = len(grid), len(grid[0])
    
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '.' and (r, c) != start_pos:
                new_grid = [list(row) for row in grid]
                new_grid[r][c] = '#'
                visited = simulate_patrol(new_grid, start_pos, start_dir)
                
                if visited:
                    potential_positions.append((r, c))
                    
    return len(potential_positions)



In [20]:
find_loop_positions(example_grid, ex_start_pos, ex_start_dir)

6

In [21]:
find_loop_positions(grid, start_pos, start_dir)

1575